In [1]:
import pyspectral as spy
from spectral import EcostressDatabase
import plotly.express as px
import pandas as pd
pd.options.plotting.backend = "plotly"
import numpy as np

In [2]:
db = EcostressDatabase('./datasets/ecostress.db')

In [3]:
s = db.get_signature(134)
px.line(x=s.x, y=s.y).update_layout(
    title_text=s.sample_name,
    xaxis_title = 'Wavelength [microns]',
    yaxis_title = 'Reflectance [%]'
)

In [4]:
s = db.get_signature(1018)
px.line(x=s.x, y=s.y).update_layout(
    title_text=s.sample_name,
    xaxis_title = 'Wavelength [microns]',
    yaxis_title = 'Reflectance [%]'
)

In [5]:
# Dry grass/ loam

# s = db.get_signature(120)
s = db.get_signature(134)
# s = db.get_signature(470)
# s = db.get_signature(669)

px.line(x=s.x, y=s.y).update_layout(
    title_text=s.sample_name,
    xaxis_title = 'Wavelength [microns]',
    yaxis_title = 'Reflectance [%]'
)

In [6]:
from pyspectral.rsr_reader import RelativeSpectralResponse


# Sentinel-3B slstr
olci = RelativeSpectralResponse('Sentinel-3B', 'slstr')

# Landsat-8 oli
# olci = RelativeSpectralResponse('Landsat-8', 'oli')

In [7]:
band_names = olci.band_names

In [8]:
all_bands = pd.DataFrame()

for band in band_names:
    rsr = olci.rsr[band]['det-1']['response']
    resp = np.where(rsr < 0.015, np.nan, rsr)
    wl_ = olci.rsr[band]['det-1']['wavelength']
    wvl = np.where(np.isnan(resp), np.nan, wl_)

    df = pd.DataFrame(resp, index=wvl)
    df['band'] = band
    # srf_dict[band] = (pd.Series(resp, index=wvl))
    all_bands = all_bands.append(df)
    all_bands.dropna()

In [9]:
fig = px.line(all_bands, facet_col='band', facet_col_wrap=3,
              facet_row_spacing=0.12, # default is 0.07 when facet_col_wrap is used
              facet_col_spacing=0.08, # default is 0.03)
              title='Sentinel-3B SLSTR Spectral Response',
              width = 900, height = 900
)
fig.update_layout(showlegend=False)
fig.update_yaxes(title="RSR [%]", col=1)
fig.update_xaxes(title = "wavelength [um]", matches=None, showticklabels=True, row=1)
fig.update_xaxes(matches=None, showticklabels=True,  row=2)
fig.update_xaxes(matches=None, showticklabels=True, row=3)